In [92]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [93]:
pd.options.display.max_colwidth = 300

In [94]:
data_path = '/Volumes/ja2/vegan/vegan_parser/scraping/ewg_data/'

In [95]:
column_names = ['datetime_pulled',
               'search_page_num',
               'chemical_name',
               'score_url',
               'data_availability',
               'chemical_concerns',
               'chemical_functions',
               'chemical_about',
               'chemical_synonyms']

In [96]:
filepaths = [os.path.join(data_path,f) for f in os.listdir(data_path) if f.startswith('chemical_details')]

In [97]:
len(filepaths)

252

In [98]:
df = pd.concat([pd.read_csv(f, names=column_names) for f in filepaths], axis=0)

In [99]:
df.shape

(9062, 9)

In [100]:
df[df['chemical_name']=='SODIUM RAPESEEDATE']

,datetime_pulled,search_page_num,chemical_name,score_url,data_availability,chemical_concerns,chemical_functions,chemical_about,chemical_synonyms
2,2019-12-09 21:55:34.303180,157,SODIUM RAPESEEDATE,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: None,NaN,"surfactant - cleansing agent, surfactant - emulsifying agent, viscosity increasing agent -nonaqueous, cleansing, emulsifying, surfactant, and viscosity controlling",Sodium Rapeseedate is a sodium salt of the fatty acids derived from Brassica Campestris(Rapeseed) Oil (q.v.).,"FATTY ACIDS, RAPESEED OIL, SODIUM SALT, RAPESEED OIL SODIUM SALT FATTY ACIDS, SODIUM RAPESEEDATE, and SODIUM SALT FATTY ACIDS, RAPESEED OIL"


In [101]:
df[df['chemical_name']=='SODIUM RAPESEEDATE']['chemical_synonyms'].values

array(['FATTY ACIDS, RAPESEED OIL, SODIUM SALT, RAPESEED OIL SODIUM SALT FATTY ACIDS, SODIUM RAPESEEDATE, and SODIUM SALT FATTY ACIDS, RAPESEED OIL'],
      dtype=object)

In [102]:
# Oh, it's not just a list, there's an "and" in there... but if the names are always capitalized, that's a way to deal with it.
# Let's look up the alternative names


In [103]:
df[df['chemical_name']=='FATTY ACIDS']

,datetime_pulled,search_page_num,chemical_name,score_url,data_availability,chemical_concerns,chemical_functions,chemical_about,chemical_synonyms
27,2019-12-09 09:41:55.692990,124,FATTY ACIDS,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: Fair,"Multiple, additive exposure sources (high)",NaN,NaN,FATTY ACIDS


In [104]:
# OH COME ON - these data are different!
# Because Sodium Rapeseedate is one of many Fatty Acids out there.

In [105]:
df[df['chemical_name']=='RAPESEED OIL']

,datetime_pulled,search_page_num,chemical_name,score_url,data_availability,chemical_concerns,chemical_functions,chemical_about,chemical_synonyms


In [106]:
# Okay, but it looks like direct synoyms are not in the list.

### A few things to do:
- duplicate rows for all the chemical synonyms, so we can have all the names in a column to look up
- translate the score_url to actual scores


In [107]:
# Translating score_url to actual scores.

df['score_url'].unique()

array(['https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=8&score_min=8',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=2&score_min=1',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=2&score_min=2',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=3&score_min=1',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=4&score_min=2',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=3&score_min=3',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=4&score_min=4',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=7&score_min=5',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=5&score_min=2',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=6&score_min=3',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=4&score_min=1',
       'https://www.ewg.org/skindeep/squircle/show.svg?score=3&score_min=2',

The scores that have a range means it depends on usage.
But, to be fair, water has a value of 1, but if you inhale too much water, or drink way too much of it, you can die. So, we should take these with a grain of salt

In [108]:
df[df['chemical_name']=='WATER']

,datetime_pulled,search_page_num,chemical_name,score_url,data_availability,chemical_concerns,chemical_functions,chemical_about,chemical_synonyms
0,2019-12-07 11:47:29.665736,1,WATER,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: Robust,NaN,solvent,NaN,"AQUA, DEIONIZED WATER, DISTILLED WATER, ONSEN-SUI, PURIFIED WATER, and WATER"


In [109]:
df.iloc[0,3]

'https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1'

In [110]:
# Let's write out the min and max scores to their own columns, and average them to get an average score
# we'll need a function to read these out

In [111]:
def extract_max(string):
    score_range = [int(s) for s in string if s.isdigit()]
    return score_range[0]

def extract_min(string):
    score_range = [int(s) for s in string if s.isdigit()]
    return score_range[1]

def extract_mean(string):
    score_range = [int(s) for s in string if s.isdigit()]
    return np.mean(score_range)

In [112]:
'111'.isdigit()

True

In [113]:
string = 'https://www.ewg.org/skindeep/squircle/show.svg?score=9&score_min=4'

In [114]:
[int(s) for s in string if s.isdigit()]

[9, 4]

In [115]:
np.mean([1, 1])

1.0

In [116]:
df['min_score'] = df['score_url'].apply(lambda x: extract_min(x))
df['max_score'] = df['score_url'].apply(lambda x: extract_max(x))
df['mean_score'] = df['score_url'].apply(lambda x: extract_mean(x))

In [117]:
pd.options.display.max_colwidth = 250

In [118]:
df['chemical_synonyms'].sample(40)

10                                                                                                                                            2-BUTENEDIOIC ACID (2Z)-, 2BUTENEDIOIC ACID (Z), CIS-1,2-ETHYLENEDICARBOXYLIC ACID, MALEIC ACID, and MALEIC ACID.
0                                                                                                                                                                                                                             ADIPIC ACID/DIGLYCOL CROSSPOLYMER
29                                                                                                                                               PEG-3 DISTEARATE, POLYETHYLENE GLYCOL (3) DISTEARATE, POLYOXYETHYLENE (3) DISTEARATE, and TRIGLYCOL DISTEARATE
4            12-HYDROXY- MONOESTER WITH TRIGLYCEROL 9-OCTADECENOIC ACID, 9-OCTADECENOIC ACID, 12-HYDROXY-, MONOESTER WITH TRIGLYCEROL, MONOESTER WITH TRIGLYCEROL 9-OCTADECENOIC ACID, 12-HYDROXY-, POLYGLYCERYL-3 RICINOLEATE, and TRIG

# After looking at a bunch of entries, it looks like each ingredient is split by ',' OR ";" OR "and".  (And sometimes, both.)
# Also noticed a few instances of spaces where there shouldn't be (e.g. "DIS ODIUM" instead of "DISODIUM"), but we can't do much about that. 
# Also, some commas are included where they are part of the name of the chemical: "1,2,3-PROPANETRIOL, 1,2,3-TRIHYDROXYPROPANE, 1,2,3PROPANETRIOL, ..."
Hopefully, we can do some fuzzy matching and catch any missing text
We'll have to ignore any numbers

In [119]:
not "1".isdigit()

False

In [120]:
def extract_synonymns_list(str1):
#     print(str1)
    if pd.isna(str1):
        return []
    
    split_and_list = str1.split('and')
    split_and_comma_list_in_list = [string.split(',') for string in split_and_list]
    split_and_comma_semi_list_in_list = [item.strip().split(';') for sublist in split_and_comma_list_in_list for item in sublist]

    intermediate_list = [item for sublist in split_and_comma_semi_list_in_list for item in sublist if item!='']
   
    # also need to filter out numbers in the ingredients due to odd splitting
    
    return [item for item in intermediate_list if not item.isdigit()]
    

In [121]:
str1 = 'PEG-75 MEADOWFOAM OIL, POLYETHYLENE GLYCOL 4000 MEADOWFOAM OIL, and POLYOXYETHYLENE (75) MEADOWFOAM OIL'
str2 = 'PEG-75 MEADOWFOAM OIL, POLYETHYLENE GLYCOL 4000 MEADOWFOAM OIL and POLYOXYETHYLENE (75) MEADOWFOAM OIL'
str3 = 'PEG-75 MEADOWFOAM OIL, POLYETHYLENE GLYCOL 4000 MEADOWFOAM OIL; and POLYOXYETHYLENE (75) MEADOWFOAM OIL'
str4 = '1, 2, 3 SOMETHING OR OTHER, WATER'


In [122]:
extract_synonymns_list(str1)

['PEG-75 MEADOWFOAM OIL',
 'POLYETHYLENE GLYCOL 4000 MEADOWFOAM OIL',
 'POLYOXYETHYLENE (75) MEADOWFOAM OIL']

In [123]:
extract_synonymns_list(str2)

['PEG-75 MEADOWFOAM OIL',
 'POLYETHYLENE GLYCOL 4000 MEADOWFOAM OIL',
 'POLYOXYETHYLENE (75) MEADOWFOAM OIL']

In [124]:
extract_synonymns_list(str3)

['PEG-75 MEADOWFOAM OIL',
 'POLYETHYLENE GLYCOL 4000 MEADOWFOAM OIL',
 'POLYOXYETHYLENE (75) MEADOWFOAM OIL']

In [125]:
extract_synonymns_list(str4)

['3 SOMETHING OR OTHER', 'WATER']

### Okay, that looks about right... 
I don't see any way out of iterating through the rows to deal with dupes.
We'll make a new dataframe and append to that.
I'm not going to bother including a value for the synonym'ed rows



In [126]:
syn_col_names = list(df.columns)+['is_syn']
syn_col_names

['datetime_pulled',
 'search_page_num',
 'chemical_name',
 'score_url',
 'data_availability',
 'chemical_concerns',
 'chemical_functions',
 'chemical_about',
 'chemical_synonyms',
 'min_score',
 'max_score',
 'mean_score',
 'is_syn']

In [127]:
#

In [132]:
print(datetime.now())

2020-01-05 22:53:48.074892


In [133]:
df_test = df.head(20)

In [134]:
df = df.reset_index(drop=True)

In [135]:
df.tail()

,datetime_pulled,search_page_num,chemical_name,score_url,data_availability,chemical_concerns,chemical_functions,chemical_about,chemical_synonyms,min_score,max_score,mean_score
9057,2019-12-09 00:08:43.920780,99,VETIVERIA ZIZANIOIDES (VETIVER),https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: Limited,NaN,NaN,NaN,VETIVERIA ZIZANIOIDES and VETIVERIA ZIZANIOIDES (VETIVER),1,1,1.0
9058,2019-12-09 00:09:21.984983,99,VIOLA TRICOLOR (HEARTSEASE) FLOWER EXTRACT,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: Limited,NaN,NaN,NaN,"HEARTSEASE FLOWER EXTRACT, VIOLA TRICOLOR (HEARTSEASE) FLOWER EXTRACT, and VIOLA TRICOLOR FLOWER EXTRACT",1,1,1.0
9059,2019-12-09 00:09:59.672816,99,RUMEX OCCIDENTALIS EXTRACT,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: None,NaN,"not reported, skin conditioning, and soothing",Rumex Occidentalis Extract is an extract of the plant Rumex occidentalis.,"EXTRACT OF RUMEX OCCIDENTALIS, RUMEX OCCIDENTALIS (RUMEX) EXTRAC, and RUMEX OCCIDENTALIS EXTRACT",1,1,1.0
9060,2019-12-09 00:10:37.660536,99,SOPHORA JAPONICA BUD EXTRACT,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: None,NaN,not reported and skin protecting,Sophora Japonica Bud Extract is an extract of the buds of Sophora japonica.,"EXTRACT OF SOPHORA JAPONICA BUD, SOPHORA JAPONICA BUD EXTRACT, and SOPHORA JAPONICA, EXT.",1,1,1.0
9061,2019-12-09 00:11:16.866515,99,BARIUM,https://www.ewg.org/skindeep/squircle/show.svg?score=3&score_min=1,Data: Fair,Use restrictions (moderate) and Non-reproductive organ system toxicity (low),NaN,NaN,"BA, BARIUM, BARIUM, DISSOLVED, and BARIUM, TOTAL",1,3,2.0


In [ ]:
# Note - it looks like the synonyms list in the scraped data includes the main chemical.
# So we don't have to worry about bringing that along

In [136]:
# OH man this is going to be SLOW AF. But, I don't care if it takes a long time (like, under an hour)

df_wsyns = pd.DataFrame([],columns=syn_col_names)  # DataFrame WithSYNonymS
print(datetime.now())

for index, row in df.iterrows():
    
    if index%100==0:
        print(f'index: {index} of {len(df)} rows')
    
    syn_list = extract_synonymns_list(row['chemical_synonyms'])
    
    if syn_list!=[]:

        # append the original
        row['is_syn'] = False
        df_wsyns = df_wsyns.append(row, ignore_index=True)
                        
        for syn in syn_list:
            row['chemical_name'] = syn        
            row['is_syn'] = True
            if syn not in df_wsyns['chemical_name'].values:
                df_wsyns = df_wsyns.append(row, ignore_index=True)

    else:
        row['is_syn'] = False
        df_wsyns = df_wsyns.append(row, ignore_index=True)
    
print(datetime.now())
df_wsyns.to_csv('dataframe_with_syns.csv', index=False)
            

2020-01-05 22:54:11.748538
index: 0 of 9062 rows
index: 100 of 9062 rows
index: 200 of 9062 rows
index: 300 of 9062 rows
index: 400 of 9062 rows
index: 500 of 9062 rows
index: 600 of 9062 rows
index: 700 of 9062 rows
index: 800 of 9062 rows
index: 900 of 9062 rows
index: 1000 of 9062 rows
index: 1100 of 9062 rows
index: 1200 of 9062 rows
index: 1300 of 9062 rows
index: 1400 of 9062 rows
index: 1500 of 9062 rows
index: 1600 of 9062 rows
index: 1700 of 9062 rows
index: 1800 of 9062 rows
index: 1900 of 9062 rows
index: 2000 of 9062 rows
index: 2100 of 9062 rows
index: 2200 of 9062 rows
index: 2300 of 9062 rows
index: 2400 of 9062 rows
index: 2500 of 9062 rows
index: 2600 of 9062 rows
index: 2700 of 9062 rows
index: 2800 of 9062 rows
index: 2900 of 9062 rows
index: 3000 of 9062 rows
index: 3100 of 9062 rows
index: 3200 of 9062 rows
index: 3300 of 9062 rows
index: 3400 of 9062 rows
index: 3500 of 9062 rows
index: 3600 of 9062 rows
index: 3700 of 9062 rows
index: 3800 of 9062 rows
index: 390

In [137]:
df_wsyns.tail()

,datetime_pulled,search_page_num,chemical_name,score_url,data_availability,chemical_concerns,chemical_functions,chemical_about,chemical_synonyms,min_score,max_score,mean_score,is_syn
34226,2019-12-09 00:10:37.660536,99,SOPHORA JAPONICA BUD EXTRACT,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: None,NaN,not reported and skin protecting,Sophora Japonica Bud Extract is an extract of the buds of Sophora japonica.,"EXTRACT OF SOPHORA JAPONICA BUD, SOPHORA JAPONICA BUD EXTRACT, and SOPHORA JAPONICA, EXT.",1,1,1.0,False
34227,2019-12-09 00:10:37.660536,99,EXTRACT OF SOPHORA JAPONICA BUD,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: None,NaN,not reported and skin protecting,Sophora Japonica Bud Extract is an extract of the buds of Sophora japonica.,"EXTRACT OF SOPHORA JAPONICA BUD, SOPHORA JAPONICA BUD EXTRACT, and SOPHORA JAPONICA, EXT.",1,1,1.0,True
34228,2019-12-09 00:10:37.660536,99,SOPHORA JAPONICA,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: None,NaN,not reported and skin protecting,Sophora Japonica Bud Extract is an extract of the buds of Sophora japonica.,"EXTRACT OF SOPHORA JAPONICA BUD, SOPHORA JAPONICA BUD EXTRACT, and SOPHORA JAPONICA, EXT.",1,1,1.0,True
34229,2019-12-09 00:11:16.866515,99,BARIUM,https://www.ewg.org/skindeep/squircle/show.svg?score=3&score_min=1,Data: Fair,Use restrictions (moderate) and Non-reproductive organ system toxicity (low),NaN,NaN,"BA, BARIUM, BARIUM, DISSOLVED, and BARIUM, TOTAL",1,3,2.0,False
34230,2019-12-09 00:11:16.866515,99,BA,https://www.ewg.org/skindeep/squircle/show.svg?score=3&score_min=1,Data: Fair,Use restrictions (moderate) and Non-reproductive organ system toxicity (low),NaN,NaN,"BA, BARIUM, BARIUM, DISSOLVED, and BARIUM, TOTAL",1,3,2.0,True


In [ ]:
# Took <15 min, that's fine.
# 34230 entries, 20MB. 

In [138]:
!tail dataframe_with_syns.csv

2019-12-09 00:09:21.984983,99,HEARTSEASE FLOWER EXTRACT,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: Limited,,,,"HEARTSEASE FLOWER EXTRACT, VIOLA TRICOLOR (HEARTSEASE) FLOWER EXTRACT, and VIOLA TRICOLOR FLOWER EXTRACT",1,1,1.0,True
2019-12-09 00:09:21.984983,99,VIOLA TRICOLOR FLOWER EXTRACT,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: Limited,,,,"HEARTSEASE FLOWER EXTRACT, VIOLA TRICOLOR (HEARTSEASE) FLOWER EXTRACT, and VIOLA TRICOLOR FLOWER EXTRACT",1,1,1.0,True
2019-12-09 00:09:59.672816,99,RUMEX OCCIDENTALIS EXTRACT,https://www.ewg.org/skindeep/squircle/show.svg?score=1&score_min=1,Data: None,,"not reported, skin conditioning, and soothing",Rumex Occidentalis Extract is an extract of the plant Rumex occidentalis.,"EXTRACT OF RUMEX OCCIDENTALIS, RUMEX OCCIDENTALIS (RUMEX) EXTRAC, and RUMEX OCCIDENTALIS EXTRACT",1,1,1.0,False
2019-12-09 00:09:59.672816,99,EXTRACT OF RUMEX OCCIDENTALIS,https://www.ewg.org/skindeep/squircle/show.sv